<a href="https://colab.research.google.com/github/reznimat/ppzd_sp/blob/main/semestralni_prace.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Načtení knihoven**

In [1]:
import csv
import pandas as pd
import math as mt

**Načtení datových souborů**

In [7]:
url_sales = 'https://raw.githubusercontent.com/reznimat/ppzd_sp/main/Online_Sales.csv'
df_sales = pd.read_csv(url_sales)

url_coupons = 'https://raw.githubusercontent.com/reznimat/ppzd_sp/main/Discount_Coupon.csv'
df_coupons = pd.read_csv(url_coupons,sep=";")

print('Sales')
print(df_sales.count())

print('------')

print('Coupons')
print(df_coupons.count())

Sales
CustomerID             52924
Transaction_ID         52924
Transaction_Date       52924
Product_SKU            52924
Product_Description    52924
Product_Category       52924
Quantity               52924
Avg_Price              52924
Delivery_Charges       52924
Coupon_Status          52924
dtype: int64
------
Coupons
Month               204
Product_Category    204
Coupon_Code         204
Discount_pct        204
dtype: int64


**Počet a výpis jedinečných produktových kategorií**

In [8]:
# Počet a výpis distinktních produktových kategorií
uniq = set()
for i, row in df_sales.iterrows():
  uniq.add(row["Product_Category"])

print("Produktových kategorií: "+ str(len(uniq)) )  # vypíše počet produktových kategorií
print(uniq)  # vypíše seznam produktových kategorií

Produktových kategorií: 20
{'Android', 'Google', 'Office', 'Notebooks & Journals', 'Bottles', 'Drinkware', 'Bags', 'Nest-Canada', 'Fun', 'More Bags', 'Nest', 'Gift Cards', 'Lifestyle', 'Backpacks', 'Housewares', 'Apparel', 'Waze', 'Headgear', 'Nest-USA', 'Accessories'}


**Suma tržeb pro kategorii Apparel** (bez započítání slev)

In [10]:
ApparelSales = 0
AvgOrderPrice = 0

for i, row in df_sales.iterrows():
  if row.Product_Category == 'Apparel':
    ApparelSales += row.Quantity*row.Avg_Price

print(ApparelSales)

591145.8000000062
